In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 10
import math as m


In [ ]:
## pip install py_vollib
## pip install py_vollib_vectorized
#https://www.youtube.com/watch?v=keN1qXTD1g4

In [2]:
from py_vollib.black_scholes_merton import black_scholes_merton
from py_vollib.black_scholes_merton.greeks.analytical import delta
from py_vollib.black_scholes_merton.greeks.analytical import vega
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility

#import py_vollib.black.implied_volatility
#import py_vollib_vectorized


In [3]:
def bsm_px(row):
    cp = row['cp']
    upx = row['upx']
    strike = row['strike']
    t2x = row['t2x']
    rf = 0  #risk free rate
    volatility = row['volatility']
    q = 0  #dividend rate
    px = black_scholes_merton(cp, upx, strike, t2x, rf, volatility, q)
    px =np.round(px, 2)
    return(px)

In [4]:
def bsm_delta(row):
    cp = row['cp']
    upx = row['upx']
    strike = row['strike']
    t2x = row['t2x']
    rf = 0  #risk free rate
    volatility = row['volatility']
    q = 0  #dividend rate
    if t2x == 0:
        return(0)
    diff = delta(cp, upx, strike, t2x, rf, volatility, q)
    diff = np.round(diff, 3)
    return(diff)

In [136]:
def bsm_vega(row):
    cp = row['cp']
    upx = row['upx']
    strike = row['strike']
    t2x = row['t2x']
    rf = 0  #risk free rate
    volatility = row['volatility']
    q = 0  #dividend rate
    if t2x == 0:
        return(0)
    vga = vega(cp, upx, strike, t2x, rf, volatility, q)
    vga = np.round(vga, 3)
    return(vga)

## Geometric Brownian motion
#price process continouse
#log return over any periods are independent
#returns during any 2 disjoint periods are independent
# delta-hedging => dynamic strategy
# vs static

In [ ]:
# C(K, T) - P(K, T) = S - Ke^(-rT)

In [6]:
#pricing_vol = implied_volatility(price = 2.25, S = 168, K = 24/252, r = 0, q = 0, flag = 'p')
#pricing_vol
#pricing_vol
# S =  strike price, t =  24 time to maturaty / 252 trading days, r= risk free rate, q= div rate, 
# flag = "p" for put
pricing_vol = implied_volatility(price = 2.25, S = 168, K = 160, t = 24/252, r = 0, q = 0, flag = 'p')
pricing_vol = np.round(pricing_vol, 4)
pricing_vol

0.2636

In [7]:
# setting the random seed
np.random.seed(1)

#parameter of simulation
r = 0
path_vol = pricing_vol
dt = 1./252

#initializing paths
single_path = np.zeros(25)
single_path[0] = 168

#looping through days and generating steps in the path

for t in range(1, 25):
    z = np.random.standard_normal(1)
    single_path[t] = single_path[t - 1] * np.exp((r - 0.5 * path_vol ** 2) * dt + path_vol * np.sqrt(dt) * z) ##memorize line
    single_path[t] = np.round(single_path[t], 2)

In [8]:
## single_path
single_path

array([168.  , 172.57, 170.8 , 169.29, 166.28, 168.66, 162.31, 167.06,
       164.94, 165.79, 165.08, 169.11, 163.4 , 162.51, 161.45, 164.5 ,
       161.5 , 161.02, 158.67, 158.76, 160.28, 157.36, 160.36, 162.76,
       164.1 ])

In [9]:
df_path = \
    (
    pd.DataFrame(
        {'underlying' :'QQQ',
         'cp' : 'p',
         'strike' :160,
         'volatility':0.2636,
         'upx' :single_path,
         'd2x':list(range(24, -1, -1)),
         'buy_sell' :1,  #controls wheth you are going LONG or SHORT!!!!!  ###### THIS case going long = positive 1
        }
    )
    .assign(t2x = lambda df: df.d2x/252)
    )
df_path

,underlying,cp,strike,volatility,upx,d2x,buy_sell,t2x
0,QQQ,p,160,0.2636,168.00,24,1,0.095238
1,QQQ,p,160,0.2636,172.57,23,1,0.091270
2,QQQ,p,160,0.2636,170.80,22,1,0.087302
3,QQQ,p,160,0.2636,169.29,21,1,0.083333
4,QQQ,p,160,0.2636,166.28,20,1,0.079365
...,...,...,...,...,...,...,...,...
20,QQQ,p,160,0.2636,160.28,4,1,0.015873
21,QQQ,p,160,0.2636,157.36,3,1,0.011905
22,QQQ,p,160,0.2636,160.36,2,1,0.007937
23,QQQ,p,160,0.2636,162.76,1,1,0.003968


In [10]:
#add option price
df_path['option price'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_px, axis = 1)
df_path

,underlying,cp,strike,volatility,upx,d2x,buy_sell,t2x,option price
0,QQQ,p,160,0.2636,168.00,24,1,0.095238,2.25
1,QQQ,p,160,0.2636,172.57,23,1,0.091270,1.21
2,QQQ,p,160,0.2636,170.80,22,1,0.087302,1.44
3,QQQ,p,160,0.2636,169.29,21,1,0.083333,1.67
4,QQQ,p,160,0.2636,166.28,20,1,0.079365,2.33
...,...,...,...,...,...,...,...,...,...
20,QQQ,p,160,0.2636,160.28,4,1,0.015873,1.98
21,QQQ,p,160,0.2636,157.36,3,1,0.011905,3.44
22,QQQ,p,160,0.2636,160.36,2,1,0.007937,1.33
23,QQQ,p,160,0.2636,162.76,1,1,0.003968,0.21


In [11]:
#put option

df_path['delta'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_delta, axis = 1)
df_path

,underlying,cp,strike,volatility,upx,d2x,buy_sell,t2x,option price,delta
0,QQQ,p,160,0.2636,168.00,24,1,0.095238,2.25,-0.261
1,QQQ,p,160,0.2636,172.57,23,1,0.091270,1.21,-0.161
2,QQQ,p,160,0.2636,170.80,22,1,0.087302,1.44,-0.190
3,QQQ,p,160,0.2636,169.29,21,1,0.083333,1.67,-0.218
4,QQQ,p,160,0.2636,166.28,20,1,0.079365,2.33,-0.289
...,...,...,...,...,...,...,...,...,...,...
20,QQQ,p,160,0.2636,160.28,4,1,0.015873,1.98,-0.472
21,QQQ,p,160,0.2636,157.36,3,1,0.011905,3.44,-0.714
22,QQQ,p,160,0.2636,160.36,2,1,0.007937,1.33,-0.457
23,QQQ,p,160,0.2636,162.76,1,1,0.003968,0.21,-0.150


In [12]:
#option PNL
df_path['option_pnl'] = df_path['buy_sell'] * df_path['option price'].diff()
df_path

,underlying,cp,strike,volatility,upx,d2x,buy_sell,t2x,option price,delta,option_pnl
0,QQQ,p,160,0.2636,168.00,24,1,0.095238,2.25,-0.261,NaN
1,QQQ,p,160,0.2636,172.57,23,1,0.091270,1.21,-0.161,-1.04
2,QQQ,p,160,0.2636,170.80,22,1,0.087302,1.44,-0.190,0.23
3,QQQ,p,160,0.2636,169.29,21,1,0.083333,1.67,-0.218,0.23
4,QQQ,p,160,0.2636,166.28,20,1,0.079365,2.33,-0.289,0.66
...,...,...,...,...,...,...,...,...,...,...,...
20,QQQ,p,160,0.2636,160.28,4,1,0.015873,1.98,-0.472,-1.05
21,QQQ,p,160,0.2636,157.36,3,1,0.011905,3.44,-0.714,1.46
22,QQQ,p,160,0.2636,160.36,2,1,0.007937,1.33,-0.457,-2.11
23,QQQ,p,160,0.2636,162.76,1,1,0.003968,0.21,-0.150,-1.12


In [13]:
df_path['delta_hedge_pnl'] = -df_path['buy_sell'] * df_path['delta'].shift(1) * df_path['upx'].diff()
#shift(1) is because of the delta held at the end of yesterday
df_path

,underlying,cp,strike,volatility,upx,d2x,buy_sell,t2x,option price,delta,option_pnl,delta_hedge_pnl
0,QQQ,p,160,0.2636,168.00,24,1,0.095238,2.25,-0.261,NaN,NaN
1,QQQ,p,160,0.2636,172.57,23,1,0.091270,1.21,-0.161,-1.04,1.19277
2,QQQ,p,160,0.2636,170.80,22,1,0.087302,1.44,-0.190,0.23,-0.28497
3,QQQ,p,160,0.2636,169.29,21,1,0.083333,1.67,-0.218,0.23,-0.28690
4,QQQ,p,160,0.2636,166.28,20,1,0.079365,2.33,-0.289,0.66,-0.65618
...,...,...,...,...,...,...,...,...,...,...,...,...
20,QQQ,p,160,0.2636,160.28,4,1,0.015873,1.98,-0.472,-1.05,0.87552
21,QQQ,p,160,0.2636,157.36,3,1,0.011905,3.44,-0.714,1.46,-1.37824
22,QQQ,p,160,0.2636,160.36,2,1,0.007937,1.33,-0.457,-2.11,2.14200
23,QQQ,p,160,0.2636,162.76,1,1,0.003968,0.21,-0.150,-1.12,1.09680


In [14]:
#total_pnl
df_path['total_pnl'] = df_path['option_pnl'] + df_path['delta_hedge_pnl']
df_path

,underlying,cp,strike,volatility,upx,d2x,buy_sell,t2x,option price,delta,option_pnl,delta_hedge_pnl,total_pnl
0,QQQ,p,160,0.2636,168.00,24,1,0.095238,2.25,-0.261,NaN,NaN,NaN
1,QQQ,p,160,0.2636,172.57,23,1,0.091270,1.21,-0.161,-1.04,1.19277,0.15277
2,QQQ,p,160,0.2636,170.80,22,1,0.087302,1.44,-0.190,0.23,-0.28497,-0.05497
3,QQQ,p,160,0.2636,169.29,21,1,0.083333,1.67,-0.218,0.23,-0.28690,-0.05690
4,QQQ,p,160,0.2636,166.28,20,1,0.079365,2.33,-0.289,0.66,-0.65618,0.00382
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,QQQ,p,160,0.2636,160.28,4,1,0.015873,1.98,-0.472,-1.05,0.87552,-0.17448
21,QQQ,p,160,0.2636,157.36,3,1,0.011905,3.44,-0.714,1.46,-1.37824,0.08176
22,QQQ,p,160,0.2636,160.36,2,1,0.007937,1.33,-0.457,-2.11,2.14200,0.03200
23,QQQ,p,160,0.2636,162.76,1,1,0.003968,0.21,-0.150,-1.12,1.09680,-0.02320


In [15]:
#cumulative PNL
df_path['total_pnl'].sum()

0.18382999999999813

In [17]:
#delta hedging

buy_sell = -1 ###(sell an option) negative
d2x = 24  #24 days to expiration
cp = 'p'
spot = 168
strike = 160
tenor = np.double(d2x)/252
option_price = 2.25
pricing_vol = implied_volatility(price = option_price, S = spot, K = strike, t = tenor, r = 0, q = 0, flag = cp)
path_vol = pricing_vol
hedge_frequency = d2x
dt = tenor /  hedge_frequency
r = 0   #risk free rate
num_paths = 1000



In [27]:
tenor

0.09523809523809523

In [28]:
hedge_frequency

24

In [18]:
dt

0.003968253968253968

In [30]:
t

24

In [81]:
r

0

In [64]:
#intialize array that will hold all the paths

multiple_paths = np.zeros((hedge_frequency + 1, num_paths))
multiple_paths

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [65]:
multiple_paths[0] =  spot
multiple_paths

array([[168., 168., 168., ..., 168., 168., 168.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [66]:
# setting the random seed
np.random.seed(1)



In [95]:
for t in range(1, hedge_frequency + 1):
    z = np.random.standard_normal(num_paths)
    multiple_paths[t] = multiple_paths[t - 1] * np.exp((r - 0.5 * path_vol ** 2) * dt + path_vol * np.sqrt(dt) * z)
    multiple_paths[t] = np.round(multiple_paths[t], 2)

multiple_paths

array([[168.  , 168.  , 168.  , ..., 168.  , 168.  , 168.  ],
       [171.49, 168.49, 168.69, ..., 165.79, 173.3 , 167.25],
       [175.96, 165.3 , 171.69, ..., 162.64, 165.91, 168.31],
       ...,
       [201.48, 171.8 , 167.94, ..., 166.27, 168.32, 186.05],
       [198.18, 173.17, 169.36, ..., 165.18, 168.43, 188.61],
       [199.12, 175.07, 167.73, ..., 166.16, 164.78, 188.73]])

In [102]:
#r - 0.5 * path_vol ** 2

In [98]:
#dt + path_vol + np.sqrt(dt) * z

In [99]:
#np.exp((r - 0.5 * path_vol ** 2) * dt + path_vol + np.sqrt(dt) * z)

In [100]:
#multiple_paths[t - 1]

In [103]:
#tenor

In [104]:
#t

In [105]:
#z

In [106]:
#hedge_frequency

In [113]:
#creating DataFrame

df_path = \
    pd.DataFrame(
        {'cp':cp,
         'strike':strike,
         'volatility':path_vol,
         'upx':multiple_paths[:, 0],
         't2x':np.linspace(tenor, 0, hedge_frequency + 1),
         'buy_sell':buy_sell,
        }
    )

In [114]:
#calculating prices, greeks, and PNLs
df_path['option price'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_px, axis = 1)
df_path['delta'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_delta, axis = 1)
df_path['option_pnl'] = df_path['buy_sell'] * df_path['option price'].diff()
df_path['delta_hedge_pnl'] = -df_path['buy_sell'] * df_path['delta'].shift(1) * df_path['upx'].diff()
df_path['total_pnl'] = df_path['option_pnl'] + df_path['delta_hedge_pnl']

# view the path
df_path

,cp,strike,volatility,upx,t2x,buy_sell,option price,delta,option_pnl,delta_hedge_pnl,total_pnl
0,p,160,0.263631,168.00,0.095238,-1,2.25,-0.261,NaN,NaN,NaN
1,p,160,0.263631,171.49,0.091270,-1,1.40,-0.181,0.85,-0.91089,-0.06089
2,p,160,0.263631,175.96,0.087302,-1,0.70,-0.104,0.70,-0.80907,-0.10907
3,p,160,0.263631,176.70,0.083333,-1,0.58,-0.090,0.12,-0.07696,0.04304
4,p,160,0.263631,180.22,0.079365,-1,0.29,-0.051,0.29,-0.31680,-0.02680
...,...,...,...,...,...,...,...,...,...,...,...
20,p,160,0.263631,199.38,0.015873,-1,0.00,-0.000,-0.00,0.00000,0.00000
21,p,160,0.263631,197.91,0.011905,-1,0.00,-0.000,-0.00,0.00000,0.00000
22,p,160,0.263631,201.48,0.007937,-1,0.00,-0.000,-0.00,-0.00000,-0.00000
23,p,160,0.263631,198.18,0.003968,-1,0.00,-0.000,-0.00,0.00000,0.00000


In [116]:
df_path["total_pnl"].sum()

-0.2286900000000009

In [123]:
#Can now generalize the above code and perform the delta_hedging calculation on each scenario. 
#Save the calculations for each scenario to analyze.

lst_scenarios = []
for ix_path in range(0, num_paths):
        
        #create dataframe
        
        df_path = \
            pd.DataFrame(
                {'cp':cp,
                 'strike':strike,
                 'volatility':path_vol,
                 'upx':multiple_paths[:, ix_path],
                 't2x':np.linspace(tenor, 0, hedge_frequency + 1),
                 'buy_sell':buy_sell,
        }
    )
        #calculating prices, greeks, and PNLs
        df_path['option price'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_px, axis = 1)
        df_path['delta'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_delta, axis = 1)
        df_path['option_pnl'] = df_path['buy_sell'] * df_path['option price'].diff()
        df_path['delta_hedge_pnl'] = -df_path['buy_sell'] * df_path['delta'].shift(1) * df_path['upx'].diff()
        df_path['total_pnl'] = df_path['option_pnl'] + df_path['delta_hedge_pnl']
        df_path['scenario'] = ix_path
        
        #storing df_path into a list
        lst_scenarios.append(df_path)
        




In [124]:
#create a single DataFrame that contains all scenarios
df_all_paths = pd.concat(lst_scenarios)

In [125]:
#view the DataFrame
df_all_paths

,cp,strike,volatility,upx,t2x,buy_sell,option price,delta,option_pnl,delta_hedge_pnl,total_pnl,scenario
0,p,160,0.263631,168.00,0.095238,-1,2.25,-0.261,NaN,NaN,NaN,0
1,p,160,0.263631,171.49,0.091270,-1,1.40,-0.181,0.85,-0.91089,-0.06089,0
2,p,160,0.263631,175.96,0.087302,-1,0.70,-0.104,0.70,-0.80907,-0.10907,0
3,p,160,0.263631,176.70,0.083333,-1,0.58,-0.090,0.12,-0.07696,0.04304,0
4,p,160,0.263631,180.22,0.079365,-1,0.29,-0.051,0.29,-0.31680,-0.02680,0
...,...,...,...,...,...,...,...,...,...,...,...,...
20,p,160,0.263631,188.16,0.015873,-1,0.00,-0.000,-0.00,0.00000,0.00000,999
21,p,160,0.263631,187.87,0.011905,-1,0.00,-0.000,-0.00,0.00000,0.00000,999
22,p,160,0.263631,186.05,0.007937,-1,0.00,-0.000,-0.00,0.00000,0.00000,999
23,p,160,0.263631,188.61,0.003968,-1,0.00,-0.000,-0.00,-0.00000,-0.00000,999


In [127]:
#use groupby to calculate cummulative PNL
df_pnl = df_all_paths.groupby(['scenario'], as_index = False)[['total_pnl']].sum()
df_pnl

,scenario,total_pnl
0,0,-0.22869
1,1,-0.26123
2,2,0.36881
3,3,-0.04960
4,4,-2.61354
...,...,...
995,995,-0.88690
996,996,0.36649
997,997,-0.62244
998,998,-0.57075


In [128]:
# Avg PNL of total_pnls is ZERO(close to zero), demonstrates the manufacturing result of the Black_scholes_Merton framework.
df_pnl['total_pnl'].mean()

0.03157087999999974

In [129]:
# is the delta-hedging reducing risk? Try to verify with data analysis ####RISK####

print(df_all_paths.groupby('scenario')['option_pnl'].sum().std())
print(df_all_paths.groupby('scenario')['total_pnl'].sum().std())

4.9533606429533075
0.7941627424962686


In [130]:
#Calc the standard deviation, min, max of cumulative PNLs.

print(np.round(df_pnl['total_pnl'].std(), 2))
print(np.round(df_pnl['total_pnl'].min(), 2))
print(np.round(df_pnl['total_pnl'].max(), 2))

0.79
-3.78
2.68


## What if realized Volatility is Different than Pricing Volatility
# add 0.05 to the pricing_vol ####example ##
# path_vol = pricing_vol  ###(original)

#path_vol = pricing_vol + 0.05   ###(+ 0.05)

In [131]:
#what if realized Volatility is Different than Pricing Volatility
# add 0.05 to the pricing_vol ####example ##
# path_vol = pricing_vol  ###(original)
#path_vol = pricing_vol + 0.05   ###(+ 0.05)

# Set simulation parameters
buy_sell = -1 ###(sell an option) negative
d2x = 24  #24 days to expiration
cp = 'p'
spot = 168
strike = 160
tenor = np.double(d2x)/252
option_price = 2.25
pricing_vol = implied_volatility(price = option_price, S = spot, K = strike, t = tenor, r = 0, q = 0, flag = cp)
path_vol = pricing_vol + 0.05 ### <-
hedge_frequency = d2x
dt = tenor /  hedge_frequency
r = 0   #risk free rate
num_paths = 1000

#initializing Paths
multiple_paths = np.zeros((hedge_frequency + 1, num_paths))
multiple_paths[0] =  spot

#setting the random seed
np.random.seed(1)

#calculating paths

for t in range(1, hedge_frequency + 1):
    z = np.random.standard_normal(num_paths)
    multiple_paths[t] = multiple_paths[t - 1] * np.exp((r - 0.5 * path_vol ** 2) * dt + path_vol * np.sqrt(dt) * z)
    multiple_paths[t] = np.round(multiple_paths[t], 2)

#perform delta-hedge calcs on all the paths
lst_scenarios = []
for ix_path in range(0, num_paths):
        
        #create dataframe
        
        df_path = \
            pd.DataFrame(
                {'cp':cp,
                 'strike':strike,
                 'volatility':path_vol,
                 'upx':multiple_paths[:, ix_path],
                 't2x':np.linspace(tenor, 0, hedge_frequency + 1),
                 'buy_sell':buy_sell,
        }
    )
        #calculating prices, greeks, and PNLs
        df_path['option price'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_px, axis = 1)
        df_path['delta'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_delta, axis = 1)
        df_path['option_pnl'] = df_path['buy_sell'] * df_path['option price'].diff()
        df_path['delta_hedge_pnl'] = -df_path['buy_sell'] * df_path['delta'].shift(1) * df_path['upx'].diff()
        df_path['total_pnl'] = df_path['option_pnl'] + df_path['delta_hedge_pnl']
        df_path['scenario'] = ix_path
        
        #storing df_path into a list
        lst_scenarios.append(df_path)


In [132]:
df_pnl['total_pnl'].mean()

0.03157087999999974

In [138]:
#is the PNL consistent with identity , anotherwords does it make sense. -negative for short,
bsm_vega(df_all_paths[['cp', 'upx', 'strike', 't2x', 'volatility']].iloc[0,:]) *5

0.8400000000000001

In [ ]:
# change the hedge frequency  d2x  variable  multiply by 2 or 4 etc...

In [139]:
#Calc the standard deviation, min, max of cumulative PNLs. ## increase number of hedge frequency STD should go down and also the magnitude.

print(np.round(df_pnl['total_pnl'].std(), 2))
print(np.round(df_pnl['total_pnl'].min(), 2))
print(np.round(df_pnl['total_pnl'].max(), 2))

0.79
-3.78
2.68


In [ ]:
#1:04 